# Building a trust-pilot scrapper

## Imports

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime as dt
import json#

/Users/kaushikdayalan/projects/storymachine_projects/trustpilot_scraper/.env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Set Trustpilot page numbers to scrape here
from_page = 300
to_page = 500

df_data = []

website_name = "www.dhl.de"

for i in range(from_page, to_page + 1):
    response = requests.get(f"https://de.trustpilot.com/review/{website_name}?page={i}")
    
    web_page = response.text
    soup = BeautifulSoup(web_page, "html.parser")
    data = json.loads(soup.find('script' , id="__NEXT_DATA__",type='application/json').text)
    data = data["props"]["pageProps"]["reviews"]
    for j, review in enumerate(data):
        reviews = []
        reviews.extend([
            review["consumer"]["displayName"],
                        review["consumer"]["countryCode"],
                        review["dates"]["publishedDate"],
                        review["title"], 
                        review["text"], 
                        review["rating"],
                        review["likes"]])
        
        if review["reply"] != None:
            reviews.append(review["reply"]["message"])
        else:
            reviews.append(None)
        df_data.append(reviews)

In [5]:
df_columns = ["reviewerName", 
              "reviewerCountryCode",
              "postDate",
              "postTitle", 
              "postContent", 
              "rating",
              "likes",
              "reply"
             ]

df = pd.DataFrame(data=df_data, columns=df_columns)

In [6]:
df["postDate"] = pd.to_datetime(df["postDate"])

In [7]:
df = df.sort_values("postDate")

In [8]:
df.isnull().sum()

reviewerName             0
reviewerCountryCode      0
postDate                 0
postTitle                0
postContent              0
rating                   0
likes                    0
reply                  460
dtype: int64

In [9]:
df.shape

(4020, 8)

,reviewerName,reviewerCountryCode,postDate,postTitle,postContent,rating,likes,reply
4019,Friederike Fromme,DE,2022-08-29 16:29:59+00:00,Am Donnerstag ein wichtiges Schreiben…,Am Donnerstag ein wichtiges Schreiben per Expr...,1,0,"Hallo, hierbei handelt es sich um Expresssendu..."
4018,Petra Wieder,DE,2022-08-29 16:47:36+00:00,In der letzten Zeit unmöglich,"In der letzten Zeit unmöglich, man bekommt ein...",1,0,"Hallo, es tut mir sehr leid, dass Sie mit der ..."
4017,Andrea Staffel,DE,2022-08-29 16:51:36+00:00,DHL - Nein danke!,... und der Stern ist noch geschenkt. Seit kna...,1,0,"Hallo, schade, dass Sie mit unserem Service ni..."
4016,Kritiker,GE,2022-08-29 16:55:46+00:00,Ein wirklich miserabler Haufen tiefer und nach...,Ein wirklich miserabler Haufen an Unfähigkeit....,1,1,Bei allem Verständnis für Ihren Ärger - so kom...
4015,Wolfgang Hager,DE,2022-08-29 17:04:53+00:00,Hat alles pünktlich und zuverlässig…,Hat alles pünktlich und zuverlässig funktionie...,5,1,"Hallo, vielen Dank für das positive Feedback, ..."
...,...,...,...,...,...,...,...,...
4,Emre Tasdemir,DE,2023-02-15 19:27:24+00:00,Schlechtester Lieferant und Kundenservice,Ich hatte ein Kopfhörer bestellt und die wurde...,1,2,"Hallo Emre Tasdemir,\nes tut uns leid, dass Si..."
3,Luana Simula,DE,2023-02-15 20:15:55+00:00,Ich hatte ein Packet als zugestellt…,"Ich hatte ein Packet als zugestellt bekommen, ...",1,0,"Hallo,\ndie geschilderten Unannehmlichkeiten b..."
2,Michael Stelzhammer,DE,2023-02-15 20:24:07+00:00,"Paketzusteller,der immer sehr…","Paketzusteller,der immer sehr zuverlässig ist,...",1,2,"Hallo Michael Stelzhammer,\nes tut uns leid zu..."
1,Ralf Hacker,DE,2023-02-15 20:30:18+00:00,Klasse,"Bis auf wenige, ärgerliche Ausnahmen, war der ...",5,0,"Hallo Ralf Hacker,\nwir freuen uns über Ihre p..."
